In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
faults = pd.read_csv("../data/J1939Faults.csv")
faults.head()

C:\Users\nitin\AppData\Local\Temp\ipykernel_64276\3681478151.py:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  faults = pd.read_csv("../data/J1939Faults.csv")


,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
0,1,990349,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,0,111,17,True,2,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25.000
1,2,990360,2015-02-21 11:34:34.000,NaN,NaN,unknown,unknown,unknown,unknown,11,629,12,True,127,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10.000
2,3,990364,2015-02-21 11:35:31.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,False,127,NaN,1369,105336226,41.421250,-87.767361,2015-02-21 11:35:26.000
3,4,990370,2015-02-21 11:35:33.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,True,127,NaN,1369,105336226,41.421018,-87.767361,2015-02-21 11:36:08.000
4,5,990416,2015-02-21 11:39:41.000,NaN,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,0,4364,17,False,2,NaN,1674,105427130,38.416481,-89.442638,2015-02-21 11:39:37.000


In [3]:
faults[(faults['spn'] == 5246)]['fmi'].unique()

array([ 0, 15, 16, 19, 14], dtype=int64)

In [4]:
#removed rows from certina long and latititue . Lets go with 80 kms using haverson.
# Locations to drop around
drop_locations = [
    (36.0666667, -86.4347222),
    (35.5883333, -86.4438888),
    (36.1950, -83.174722)
]

from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the distance in kilometers between two points on Earth 
    specified by their longitudes and latitudes in decimal degrees.
    """
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371 
    return c * r

# Mark rows to drop
radius_km = 30 #50 miles radius
def is_near_any(row, locations, radius):
    for lat, lon in locations:
        if haversine(row['Latitude'], row['Longitude'], lat, lon) <= radius:
            return True
    return False

# Apply and filter
faults['drop'] = faults.apply(lambda row: is_near_any(row, drop_locations, radius_km), axis=1)
faults_cleaned = faults[~faults['drop']].drop(columns='drop')

In [5]:
faults_cleaned

,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
0,1,990349,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,0,111,17,True,2,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25.000
1,2,990360,2015-02-21 11:34:34.000,NaN,NaN,unknown,unknown,unknown,unknown,11,629,12,True,127,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10.000
2,3,990364,2015-02-21 11:35:31.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,False,127,NaN,1369,105336226,41.421250,-87.767361,2015-02-21 11:35:26.000
3,4,990370,2015-02-21 11:35:33.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,True,127,NaN,1369,105336226,41.421018,-87.767361,2015-02-21 11:36:08.000
4,5,990416,2015-02-21 11:39:41.000,NaN,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,0,4364,17,False,2,NaN,1674,105427130,38.416481,-89.442638,2015-02-21 11:39:37.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1187330,1248454,123904424,2020-03-06 14:00:26.000,Low (Severity Low) Catalyst Tank Level,NaN,04384413*22383729*082218154102*60701732*G1*BGT*,80156139,6X1u17D1500000000,CMMNS,0,1761,17,False,3,NaN,2282,105439740,37.094768,-85.897407,2020-03-06 14:00:21.000
1187331,1248455,123905139,2020-03-06 14:04:23.000,Condition Exists Engine Protection Torque Derate,NaN,04358814*06099720*030816202706*09400153*G1*BDR*,79932020,6X1u13D1500000000,CMMNS,0,1569,31,True,5,NaN,1994,105354084,34.390740,-79.461805,2020-03-06 14:04:59.000
1187332,1248456,123905996,2020-03-06 14:13:38.000,Abnormal Rate of Change Aftertreatment 1 Intak...,NaN,05317106*05100987*050719120655*09401585*G1*BDR*,79880653,6X1u13D1500000000,CMMNS,0,3216,10,True,1,NaN,1850,105336308,34.430370,-84.920509,2020-03-06 14:14:14.000
1187333,1248457,123906113,2020-03-06 14:14:13.000,Low (Severity Medium) Engine Coolant Level,NaN,04384413*22544852*090619141107*60701756*G1*BGT*,NaN,NaN,NaN,0,111,18,True,8,NaN,2377,108605700,35.030925,-85.321527,2020-03-06 14:14:49.000


In [6]:
faults_selected = faults_cleaned[['RecordID','EventTimeStamp','eventDescription','spn','fmi','active','activeTransitionCount','EquipmentID','Latitude','Longitude','LocationTimeStamp']]
faults_selected.head()

,RecordID,EventTimeStamp,eventDescription,spn,fmi,active,activeTransitionCount,EquipmentID,Latitude,Longitude,LocationTimeStamp
0,1,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,111,17,True,2,1439,38.857638,-84.626851,2015-02-21 11:34:25.000
1,2,2015-02-21 11:34:34.000,NaN,629,12,True,127,1439,38.857638,-84.626851,2015-02-21 11:35:10.000
2,3,2015-02-21 11:35:31.000,Incorrect Data Steering Wheel Angle,1807,2,False,127,1369,41.421250,-87.767361,2015-02-21 11:35:26.000
3,4,2015-02-21 11:35:33.000,Incorrect Data Steering Wheel Angle,1807,2,True,127,1369,41.421018,-87.767361,2015-02-21 11:36:08.000
4,5,2015-02-21 11:39:41.000,NaN,4364,17,False,2,1674,38.416481,-89.442638,2015-02-21 11:39:37.000


In [7]:
faults_cleaned[(faults_selected['EquipmentID'] == 1331)]['spn'].unique()

array([50353,   596,   101], dtype=int64)

In [8]:
diagnostics = pd.read_csv("../data/VehicleDiagnosticOnboardData.csv")

diagnostics.head()

,Id,Name,Value,FaultId
0,1,IgnStatus,False,1
1,2,EngineOilPressure,0,1
2,3,EngineOilTemperature,96.74375,1
3,4,TurboBoostPressure,0,1
4,5,EngineLoad,11,1


In [9]:
diag_pivoted = diagnostics.pivot(index='FaultId', columns='Name', values='Value').reset_index()

In [10]:
diag_pivoted

Name,FaultId,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,...,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
0,1,0,14.21,False,66.48672,423178.7,100.4,11,0,96.74375,...,NaN,False,78.8,1023,True,NaN,0,3276.75,NaN,0
1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
3,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,16639,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1187330,1248454,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1023,NaN,NaN,NaN,NaN,NaN,NaN
1187331,1248455,100,14.5,True,64.6226,423937.9,185,51,37.12,211.4937,...,32,True,98.6,18431,False,NaN,65.01096,NaN,73.2,7.83
1187332,1248456,0,14.355,True,66.48672,465925.4,186.8,62,41.18,212.8438,...,NaN,True,91.4,17407,NaN,NaN,66.5741,NaN,100,6.96
1187333,1248457,1.6,14.4275,False,67.72946,28606.65625,181.4,0,27.26,221.7312,...,NaN,True,100.4,1023,False,NaN,11.84489,14.1,100,1.74


In [11]:
#Lets merge faults and diagnosis
faults_diag_merged = pd.merge(faults_selected, diag_pivoted, how='left', left_on='RecordID', right_on='FaultId')
faults_diag_merged

,RecordID,EventTimeStamp,eventDescription,spn,fmi,active,activeTransitionCount,EquipmentID,Latitude,Longitude,...,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
0,1,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,111,17,True,2,1439,38.857638,-84.626851,...,NaN,False,78.8,1023,True,NaN,0,3276.75,NaN,0
1,2,2015-02-21 11:34:34.000,NaN,629,12,True,127,1439,38.857638,-84.626851,...,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
2,3,2015-02-21 11:35:31.000,Incorrect Data Steering Wheel Angle,1807,2,False,127,1369,41.421250,-87.767361,...,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
3,4,2015-02-21 11:35:33.000,Incorrect Data Steering Wheel Angle,1807,2,True,127,1369,41.421018,-87.767361,...,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2015-02-21 11:39:41.000,NaN,4364,17,False,2,1674,38.416481,-89.442638,...,NaN,NaN,NaN,16639,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
917073,1248454,2020-03-06 14:00:26.000,Low (Severity Low) Catalyst Tank Level,1761,17,False,3,2282,37.094768,-85.897407,...,NaN,NaN,NaN,1023,NaN,NaN,NaN,NaN,NaN,NaN
917074,1248455,2020-03-06 14:04:23.000,Condition Exists Engine Protection Torque Derate,1569,31,True,5,1994,34.390740,-79.461805,...,32,True,98.6,18431,False,NaN,65.01096,NaN,73.2,7.83
917075,1248456,2020-03-06 14:13:38.000,Abnormal Rate of Change Aftertreatment 1 Intak...,3216,10,True,1,1850,34.430370,-84.920509,...,NaN,True,91.4,17407,NaN,NaN,66.5741,NaN,100,6.96
917076,1248457,2020-03-06 14:14:13.000,Low (Severity Medium) Engine Coolant Level,111,18,True,8,2377,35.030925,-85.321527,...,NaN,True,100.4,1023,False,NaN,11.84489,14.1,100,1.74


In [12]:
#convert time stamp from object to date
faults_diag_merged['EventTimeStamp'] = pd.to_datetime(faults_diag_merged['EventTimeStamp'],errors='coerce')
faults_diag_merged['LocationTimeStamp'] = pd.to_datetime(faults_diag_merged['LocationTimeStamp'],errors='coerce')
faults_diag_merged

,RecordID,EventTimeStamp,eventDescription,spn,fmi,active,activeTransitionCount,EquipmentID,Latitude,Longitude,...,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
0,1,2015-02-21 10:47:13,Low (Severity Low) Engine Coolant Level,111,17,True,2,1439,38.857638,-84.626851,...,NaN,False,78.8,1023,True,NaN,0,3276.75,NaN,0
1,2,2015-02-21 11:34:34,NaN,629,12,True,127,1439,38.857638,-84.626851,...,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
2,3,2015-02-21 11:35:31,Incorrect Data Steering Wheel Angle,1807,2,False,127,1369,41.421250,-87.767361,...,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
3,4,2015-02-21 11:35:33,Incorrect Data Steering Wheel Angle,1807,2,True,127,1369,41.421018,-87.767361,...,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2015-02-21 11:39:41,NaN,4364,17,False,2,1674,38.416481,-89.442638,...,NaN,NaN,NaN,16639,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
917073,1248454,2020-03-06 14:00:26,Low (Severity Low) Catalyst Tank Level,1761,17,False,3,2282,37.094768,-85.897407,...,NaN,NaN,NaN,1023,NaN,NaN,NaN,NaN,NaN,NaN
917074,1248455,2020-03-06 14:04:23,Condition Exists Engine Protection Torque Derate,1569,31,True,5,1994,34.390740,-79.461805,...,32,True,98.6,18431,False,NaN,65.01096,NaN,73.2,7.83
917075,1248456,2020-03-06 14:13:38,Abnormal Rate of Change Aftertreatment 1 Intak...,3216,10,True,1,1850,34.430370,-84.920509,...,NaN,True,91.4,17407,NaN,NaN,66.5741,NaN,100,6.96
917076,1248457,2020-03-06 14:14:13,Low (Severity Medium) Engine Coolant Level,111,18,True,8,2377,35.030925,-85.321527,...,NaN,True,100.4,1023,False,NaN,11.84489,14.1,100,1.74


In [13]:
# faults_targetted = faults_diag_merged[faults_diag_merged['EquipmentID']
# .isin(list(faults_diag_merged[ (faults_diag_merged['spn'] == 5246) & 
#            (faults_diag_merged['active'] == True) &
#            (faults_diag_merged['EventTimeStamp'] <= '2018-12-31')]['EquipmentID']
#            .unique()))]
# Lets take data before 12/31/2018 as training data and build on top of that
faults_targetted = faults_diag_merged[(faults_diag_merged['EventTimeStamp'] <= '2018-12-31')]

# Sort just to be safe
temp_sort_df = faults_targetted.sort_values(by=['EquipmentID', 'EventTimeStamp'])

# List to collect all event frames
all_events = []

# Loop through each TruckID
for truck_id, truck_df in temp_sort_df.groupby('EquipmentID'):
    truck_df = truck_df.sort_values('EventTimeStamp')
    
    # Find 5246 occurrences
    hits_5246 = truck_df[truck_df['spn'] == 5246]['EventTimeStamp'].reset_index(drop=True)
    
    event_counter = 1  # Reset counter for each truck

    for i, current_hit_time in enumerate(hits_5246):
        # Window: last 24 hours including current hit
        window_start = current_hit_time - pd.Timedelta(hours=14)

        # Find if any previous 5246 in that window
        previous_5246 = hits_5246[hits_5246 < current_hit_time]
        previous_5246_in_window = previous_5246[previous_5246 >= window_start]

        if not previous_5246_in_window.empty:
            # If previous 5246 found within window, start from that
            adjusted_start_time = previous_5246_in_window.max()
        else:
            # Otherwise normal 24-hour window
            adjusted_start_time = window_start
        
        mask = (truck_df['EventTimeStamp'] >= adjusted_start_time) & (truck_df['EventTimeStamp'] <= current_hit_time) & (truck_df['active'] == True)
        window_df = truck_df[mask].copy()

        # Drop if only 5246 exists in window
        if window_df['spn'].nunique() == 1 and (5246 in window_df['spn'].unique()):
            # Only 5246 present → Skip this window
            continue

        # Sort by Time just to be safe
        window_df = window_df.sort_values('EventTimeStamp')

        # De-duplication: Check (spn, Latitude, Longitude) against ALL previous
        seen = set()
        deduped_rows = []

        for idx, row in window_df.iterrows():
            key = (row['spn'], row['Latitude'], row['Longitude'])
            if key not in seen:
                seen.add(key)
                deduped_rows.append(row)

        # Create cleaned DataFrame
        deduped_window_df = pd.DataFrame(deduped_rows)
        
        # Label the Event
        deduped_window_df['Event'] = f"{truck_id}_5246_Event#{event_counter}"
        
        all_events.append(deduped_window_df)
        
        event_counter += 1

# Combine all events together
final_df = pd.concat(all_events, ignore_index=True)

# Drop exact duplicates across events
final_df = final_df.drop_duplicates(subset=['EventTimeStamp', 'spn', 'Latitude', 'Longitude'])

# Get the 5246 times for each Event
event_5246_times = final_df[final_df['spn'] == 5246].groupby('Event')['EventTimeStamp'].min()

# Merge that info back into final_df
final_df['Event_5246_Time'] = final_df['Event'].map(event_5246_times)

# Now calculate Hours Before 5246
final_df['Hours_Before_5246'] = (final_df['Event_5246_Time'] - final_df['EventTimeStamp']).dt.total_seconds() / 3600


# print("\nFinal Data with Event Labels:")
# #print(final_df)

# # # Now Pivot: Create summary table
# pivot_wider = final_df.pivot_table(
#     index='Event',
#     columns='spn',
#     values='EquipmentID',  # any non-null field works
#     aggfunc='count',
#     fill_value=0
# )

# Group by TruckID, Event, and Code
# summary_df = final_df.groupby(['EquipmentID', 'Event', 'spn']).size().reset_index(name='Count')
# summary_df

In [14]:
truck_df[truck_df['spn'] == 5246]['EventTimeStamp'].reset_index(drop=True)


Series([], Name: EventTimeStamp, dtype: datetime64[ns])

In [15]:
final_df

,RecordID,EventTimeStamp,eventDescription,spn,fmi,active,activeTransitionCount,EquipmentID,Latitude,Longitude,...,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,Event,Event_5246_Time,Hours_Before_5246
0,383326.0,2016-02-14 23:23:02,Incorrect Data J1939 Network #4,1668.0,2.0,True,1.0,304,38.099537,-85.700787,...,17407,False,NaN,63.61288,NaN,19.2,6.67,304_5246_Event#1,2016-02-15 10:59:28,11.607222
1,383327.0,2016-02-14 23:30:33,Error in System Engine Exhaust Gas Recirculati...,3821.0,11.0,True,1.0,304,38.196527,-85.733611,...,17407,False,NaN,50.85535,NaN,36,6.67,304_5246_Event#1,2016-02-15 10:59:28,11.481944
2,383328.0,2016-02-14 23:32:29,Data May Be Invalid Catalyst Tank Level,1761.0,19.0,True,12.0,304,38.212129,-85.752824,...,255,False,NaN,54.24862,NaN,0,2.61,304_5246_Event#1,2016-02-15 10:59:28,11.449722
3,383329.0,2016-02-14 23:33:13,NaN,5848.0,9.0,True,1.0,304,38.221203,-85.751388,...,17407,False,NaN,51.99129,NaN,16.8,14.5,304_5246_Event#1,2016-02-15 10:59:28,11.437500
4,383332.0,2016-02-14 23:40:53,Abnormal Update Rate Particulate Trap Lamp Com...,3697.0,9.0,True,34.0,304,38.294305,-85.757083,...,255,False,NaN,48.899,NaN,22.4,16.82,304_5246_Event#1,2016-02-15 10:59:28,11.309722
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2009,1029561.0,2018-06-25 07:30:00,NaN,4349.0,7.0,True,1.0,305,38.227870,-85.751527,...,18431,False,NaN,58.71472,NaN,0,17.98,305_5246_Event#1,2015-12-14 18:25:46,-22165.070556
2010,1029584.0,2018-06-25 08:30:02,Error in System Engine Torque Limit Request - ...,1787.0,11.0,True,1.0,305,38.222916,-85.784120,...,18431,True,NaN,0,NaN,0,0.29,305_5246_Event#1,2015-12-14 18:25:46,-22166.071111
2011,1029681.0,2018-06-25 12:12:13,High (Severity Medium) Aftertreatment 1 Outlet...,3226.0,16.0,True,3.0,305,37.784768,-85.401111,...,18431,False,NaN,52.40878,NaN,97.6,20.01,305_5246_Event#1,2015-12-14 18:25:46,-22169.774167
2012,1029757.0,2018-06-25 14:47:46,Special Instructions Maximum Vehicle Speed Limit,74.0,14.0,True,1.0,305,37.644166,-85.348148,...,22527,False,NaN,NaN,NaN,NaN,NaN,305_5246_Event#1,2015-12-14 18:25:46,-22172.366667


In [16]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1803 entries, 0 to 2013
Data columns (total 39 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   RecordID                   1803 non-null   float64       
 1   EventTimeStamp             1803 non-null   datetime64[ns]
 2   eventDescription           1266 non-null   object        
 3   spn                        1803 non-null   float64       
 4   fmi                        1803 non-null   float64       
 5   active                     1803 non-null   object        
 6   activeTransitionCount      1803 non-null   float64       
 7   EquipmentID                1803 non-null   object        
 8   Latitude                   1803 non-null   float64       
 9   Longitude                  1803 non-null   float64       
 10  LocationTimeStamp          1803 non-null   datetime64[ns]
 11  FaultId                    1803 non-null   float64       
 12  Accelerator

In [17]:
final_df

,RecordID,EventTimeStamp,eventDescription,spn,fmi,active,activeTransitionCount,EquipmentID,Latitude,Longitude,...,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,Event,Event_5246_Time,Hours_Before_5246
0,383326.0,2016-02-14 23:23:02,Incorrect Data J1939 Network #4,1668.0,2.0,True,1.0,304,38.099537,-85.700787,...,17407,False,NaN,63.61288,NaN,19.2,6.67,304_5246_Event#1,2016-02-15 10:59:28,11.607222
1,383327.0,2016-02-14 23:30:33,Error in System Engine Exhaust Gas Recirculati...,3821.0,11.0,True,1.0,304,38.196527,-85.733611,...,17407,False,NaN,50.85535,NaN,36,6.67,304_5246_Event#1,2016-02-15 10:59:28,11.481944
2,383328.0,2016-02-14 23:32:29,Data May Be Invalid Catalyst Tank Level,1761.0,19.0,True,12.0,304,38.212129,-85.752824,...,255,False,NaN,54.24862,NaN,0,2.61,304_5246_Event#1,2016-02-15 10:59:28,11.449722
3,383329.0,2016-02-14 23:33:13,NaN,5848.0,9.0,True,1.0,304,38.221203,-85.751388,...,17407,False,NaN,51.99129,NaN,16.8,14.5,304_5246_Event#1,2016-02-15 10:59:28,11.437500
4,383332.0,2016-02-14 23:40:53,Abnormal Update Rate Particulate Trap Lamp Com...,3697.0,9.0,True,34.0,304,38.294305,-85.757083,...,255,False,NaN,48.899,NaN,22.4,16.82,304_5246_Event#1,2016-02-15 10:59:28,11.309722
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2009,1029561.0,2018-06-25 07:30:00,NaN,4349.0,7.0,True,1.0,305,38.227870,-85.751527,...,18431,False,NaN,58.71472,NaN,0,17.98,305_5246_Event#1,2015-12-14 18:25:46,-22165.070556
2010,1029584.0,2018-06-25 08:30:02,Error in System Engine Torque Limit Request - ...,1787.0,11.0,True,1.0,305,38.222916,-85.784120,...,18431,True,NaN,0,NaN,0,0.29,305_5246_Event#1,2015-12-14 18:25:46,-22166.071111
2011,1029681.0,2018-06-25 12:12:13,High (Severity Medium) Aftertreatment 1 Outlet...,3226.0,16.0,True,3.0,305,37.784768,-85.401111,...,18431,False,NaN,52.40878,NaN,97.6,20.01,305_5246_Event#1,2015-12-14 18:25:46,-22169.774167
2012,1029757.0,2018-06-25 14:47:46,Special Instructions Maximum Vehicle Speed Limit,74.0,14.0,True,1.0,305,37.644166,-85.348148,...,22527,False,NaN,NaN,NaN,NaN,NaN,305_5246_Event#1,2015-12-14 18:25:46,-22172.366667


In [18]:
summary_df = final_df.groupby(['EquipmentID', 'Event', 'spn','fmi']).size().reset_index(name='Count')
summary_df

,EquipmentID,Event,spn,fmi,Count
0,304,304_5246_Event#1,1668.0,2.0,2
1,304,304_5246_Event#1,1761.0,14.0,1
2,304,304_5246_Event#1,1761.0,19.0,4
3,304,304_5246_Event#1,1787.0,11.0,1
4,304,304_5246_Event#1,3246.0,3.0,1
...,...,...,...,...,...
1329,305,305_5246_Event#1,3226.0,16.0,1
1330,305,305_5246_Event#1,4349.0,7.0,1
1331,305,305_5246_Event#1,4765.0,19.0,2
1332,305,305_5246_Event#1,4794.0,31.0,1


## Lets find the early warning spn where they appeared from 2 hours to 12 hours prior to 5246

In [19]:
# Only keep codes that happened BEFORE 5246 (not after)
early_warnings = final_df[(final_df['spn'] != 5246) & (final_df['Hours_Before_5246'] >= 2) & (final_df['Hours_Before_5246'] <= 14)]

# Group by Code
early_indicator_stats = early_warnings.groupby('spn').agg(
    avg_hours_before_5246=('Hours_Before_5246', 'mean'),
    count_occurrences=('Hours_Before_5246', 'count')
).reset_index()

# Sort to find best early indicators:
# First by count (descending), then by avg_hours (descending)
early_indicator_stats = early_indicator_stats.sort_values(
    by=['avg_hours_before_5246','count_occurrences'],
    ascending=[False, False]
)

early_indicator_stats.head()

,spn,avg_hours_before_5246,count_occurrences
17,791.0,12.948333,1
19,907.0,12.948333,1
3,94.0,12.408611,2
24,1668.0,11.607222,1
18,829.0,11.143388,96


In [20]:
early_warnings

,RecordID,EventTimeStamp,eventDescription,spn,fmi,active,activeTransitionCount,EquipmentID,Latitude,Longitude,...,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,Event,Event_5246_Time,Hours_Before_5246
0,383326.0,2016-02-14 23:23:02,Incorrect Data J1939 Network #4,1668.0,2.0,True,1.0,304,38.099537,-85.700787,...,17407,False,NaN,63.61288,NaN,19.2,6.67,304_5246_Event#1,2016-02-15 10:59:28,11.607222
1,383327.0,2016-02-14 23:30:33,Error in System Engine Exhaust Gas Recirculati...,3821.0,11.0,True,1.0,304,38.196527,-85.733611,...,17407,False,NaN,50.85535,NaN,36,6.67,304_5246_Event#1,2016-02-15 10:59:28,11.481944
2,383328.0,2016-02-14 23:32:29,Data May Be Invalid Catalyst Tank Level,1761.0,19.0,True,12.0,304,38.212129,-85.752824,...,255,False,NaN,54.24862,NaN,0,2.61,304_5246_Event#1,2016-02-15 10:59:28,11.449722
3,383329.0,2016-02-14 23:33:13,NaN,5848.0,9.0,True,1.0,304,38.221203,-85.751388,...,17407,False,NaN,51.99129,NaN,16.8,14.5,304_5246_Event#1,2016-02-15 10:59:28,11.437500
4,383332.0,2016-02-14 23:40:53,Abnormal Update Rate Particulate Trap Lamp Com...,3697.0,9.0,True,34.0,304,38.294305,-85.757083,...,255,False,NaN,48.899,NaN,22.4,16.82,304_5246_Event#1,2016-02-15 10:59:28,11.309722
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1971,1043905.0,2018-08-08 08:39:43,Abnormal Update Rate Catalyst Tank Level,1761.0,9.0,True,1.0,1921,27.461064,-82.549675,...,18431,True,NaN,0,NaN,100,0.29,1921_5246_Event#1,2018-08-08 17:55:19,9.260000
1972,1043906.0,2018-08-08 08:39:43,Abnormal Update Rate Catalyst Tank Temperature,3031.0,9.0,True,1.0,1921,27.461064,-82.549675,...,18431,True,NaN,0,NaN,100,0.29,1921_5246_Event#1,2018-08-08 17:55:19,9.260000
1973,1043907.0,2018-08-08 08:39:43,Abnormal Update Rate Catalyst Tank Reagent Qua...,3364.0,9.0,True,1.0,1921,27.461064,-82.549675,...,18431,True,NaN,0,NaN,100,0.29,1921_5246_Event#1,2018-08-08 17:55:19,9.260000
1974,1043959.0,2018-08-08 09:58:06,Condition Exists Engine Protection Torque Derate,1569.0,31.0,True,1.0,1921,27.529722,-82.507638,...,18431,False,NaN,0,NaN,100,0.58,1921_5246_Event#1,2018-08-08 17:55:19,7.953611


In [21]:
#top 5 spn codes per event

# Step 1: Group by TruckID, Event, and Code, and count
event_code_counts = early_warnings.groupby(['EquipmentID', 'Event', 'spn']).size().reset_index(name='Count')

# Step 2: For each Event, get Top 5 Codes
# Sort first by TruckID, Event, Count descending
event_code_counts = event_code_counts.sort_values(['EquipmentID', 'Event', 'Count'], ascending=[True, True, False])

# Step 3: Rank codes within each event
event_code_counts['Rank'] = event_code_counts.groupby(['EquipmentID', 'Event'])['Count'].rank(method='first', ascending=False)

# Step 4: Filter only top 5 per Event
top_codes_per_event = event_code_counts[event_code_counts['Rank'] <= 10]

top_codes_per_event

,EquipmentID,Event,spn,Count,Rank
1,304,304_5246_Event#1,1761.0,5,1.0
4,304,304_5246_Event#1,3821.0,4,2.0
3,304,304_5246_Event#1,3697.0,2,3.0
0,304,304_5246_Event#1,1668.0,1,4.0
2,304,304_5246_Event#1,1787.0,1,5.0
...,...,...,...,...,...
320,1921,1921_5246_Event#1,1569.0,1,1.0
321,1921,1921_5246_Event#1,1761.0,1,2.0
322,1921,1921_5246_Event#1,3031.0,1,3.0
323,1921,1921_5246_Event#1,3364.0,1,4.0


## Lets join spn-fmi combination to the service fault code  description

In [22]:
spn_fmi_df = early_warnings[(early_warnings['spn'] != 5246)][['spn','fmi']].value_counts().reset_index(name='Count').rename(columns={'index': 'spn'}).sort_values(by='Count', ascending=False)
spn_fmi_df = spn_fmi_df.astype('int32')
spn_fmi_df = spn_fmi_df.drop('Count',axis=1)
spn_fmi_df.drop_duplicates()
spn_fmi_df

,spn,fmi
0,96,3
1,1569,31
2,829,3
3,111,17
4,3362,31
...,...,...
62,641,11
63,639,9
64,639,2
65,611,14


In [23]:
sfc = pd.read_excel("../data/Service Fault Codes_1_0_0_167.xlsx")
sfc2 = sfc[['SPN','J1939 FMI','Lamp Color','Lamp Device','Cummins Description']]
sfc2 = sfc2.rename(columns={"SPN": "spn", "J1939 FMI": "fmi"})
sfc2

C:\Users\nitin\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,spn,fmi,Lamp Color,Lamp Device,Cummins Description
0,629,12,Red,Stop / Shutdown,Engine Control Module Critical Internal Failur...
1,635,7,Red,Stop / Shutdown,Engine Timing Actuator Driver Circuit - Mechan...
2,635,3,Amber,Warning,Engine Timing Actuator Driver Circuit - Voltag...
3,635,4,Amber,Warning,Engine Timing Actuator Driver Circuit - Voltag...
4,612,2,Red,Stop / Shutdown,Engine Magnetic Speed/Position Lost Both of Tw...
...,...,...,...,...,...
7119,524286,31,Amber,Warning,Reserved for temporary use - Condition Exists
7120,524286,31,Amber,Warning,Reserved for temporary use - Condition Exists
7121,524286,31,Amber,Warning,Reserved for temporary use - Condition Exists
7122,524286,31,Amber,Warning,Reserved for temporary use - Condition Exists


## Lets remove the spn -fmi combincation which are low and maintainance level
Remove spn = 524287,fmi=31,Amber,Warning,Reserved for future use - Condition Exists, 
dropped all maintainance level codes

In [24]:
sfc2 = sfc2[~((sfc2['Lamp Color'] == 'Maintenance'))]
#sfc2.to_csv('newspn.csv',index=False)
sfc2 = sfc2[~(sfc2['spn'].isin([524286,524287,520290]))]
sfc2

,spn,fmi,Lamp Color,Lamp Device,Cummins Description
0,629,12,Red,Stop / Shutdown,Engine Control Module Critical Internal Failur...
1,635,7,Red,Stop / Shutdown,Engine Timing Actuator Driver Circuit - Mechan...
2,635,3,Amber,Warning,Engine Timing Actuator Driver Circuit - Voltag...
3,635,4,Amber,Warning,Engine Timing Actuator Driver Circuit - Voltag...
4,612,2,Red,Stop / Shutdown,Engine Magnetic Speed/Position Lost Both of Tw...
...,...,...,...,...,...
6399,609,12,Red,Stop / Shutdown,Fuel Control Module - Bad Intelligent Device o...
6400,3597,2,Amber,Warning,Fuel Control Module Power Lost With Ignition O...
6401,168,18,Amber,Warning,Fuel Control Module Battery 1 Voltage - Data V...
6402,2791,15,NaN,NaN,EGR Valve Control Circuit Calculated Over Temp...


In [25]:
sfc2 = sfc2.merge(spn_fmi_df, how='inner', on=['spn','fmi'])
#sfc2.to_csv('spnfmi.csv', index=False)
sfc2

,spn,fmi,Lamp Color,Lamp Device,Cummins Description
0,102,4,Amber,Warning,Intake Manifold 1 Pressure Sensor Circuit - Vo...
1,91,3,Red,Stop / Shutdown,Accelerator Pedal or Lever Position Sensor 1 C...
2,639,2,Amber,Warning,"J1939 Network #1 - Data Erratic, Intermittent,..."
3,111,18,Amber,Warning,Coolant Level - Data Valid But Below Normal Op...
4,111,1,Red,Stop / Shutdown,Coolant Level - Data Valid But Below Normal Op...
...,...,...,...,...,...
87,3364,18,NaN,NaN,Aftertreatment Diesel Exhaust Fluid Quality - ...
88,245,9,NaN,NaN,Total Vehicle Distance - Abnormal Update Rate
89,251,9,NaN,NaN,Real Time Clock - Abnormal Update Rate
90,4094,18,Amber,Warning,NOx Limits Exceeded Due to Insufficient Reagen...


In [26]:
#lets check spn 111, seems not critical
sfc2[sfc2['spn'] == 111]

,spn,fmi,Lamp Color,Lamp Device,Cummins Description
3,111,18,Amber,Warning,Coolant Level - Data Valid But Below Normal Op...
4,111,1,Red,Stop / Shutdown,Coolant Level - Data Valid But Below Normal Op...
35,111,18,NaN,NaN,Coolant Level - Data Valid But Below Normal Op...
69,111,17,Amber,Warning,Coolant Level - Data Valid But Below Normal Op...


In [27]:
early_warnings[(early_warnings['spn'] != 5246)][['spn']].value_counts().reset_index(name='Count').rename(columns={'index': 'spn'}).sort_values(by='Count', ascending=False).head()

,spn,Count
0,96.0,155
1,1569.0,138
2,829.0,96
3,111.0,62
4,1761.0,25


### Lets pass this unquie spm and fmi combination to main set and mark them as derate indicators.

In [28]:
derate_codes = sfc2[['spn','fmi']].drop_duplicates()
derate_codes.sort_values(by='spn',ascending=True)
#drop spn 111 and fmi 17,18
derate_codes = derate_codes[~((derate_codes['spn'] == 111) & (derate_codes['fmi'].isin([17,18])))]

### Grab the selected columns for target data sets

In [29]:
faults_target_data = faults_targetted[['RecordID','EquipmentID','EventTimeStamp','spn','fmi','active','AcceleratorPedal','BarometricPressure','CruiseControlSetSpeed',
'EngineCoolantTemperature','EngineLoad','EngineOilPressure','EngineOilTemperature','EngineRpm','EngineTimeLtd','FuelLevel','FuelLtd','FuelTemperature',
'IgnStatus','IntakeManifoldTemperature','LampStatus','ServiceDistance','Speed','SwitchedBatteryVoltage','Throttle','TurboBoostPressure']]

In [30]:
#faults_target_data.isnull().sum()
#faults_target_data[['RecordID','EquipmentID','EventTimeStamp','spn','fmi','active']]
#faults_target_data.info()
#faults_targetted['EngineLoad'].unique()

### just check few engine data parameters

In [31]:
#derate_codes
derate_codes.to_csv('derate_codes.csv', index=False)

### Here we will take each row from target data set where event date is < 12/30/2018 and based on the derate_codes, create new column ind_derate = 1 OR 0

In [32]:
def is_derate_codes(row, derate_codes):
    if((row['spn'] in list(derate_codes['spn'].values)) &
       (row['fmi'] in list(derate_codes[derate_codes['spn'] == row['spn']]['fmi'])) &
       (row['active'] == True)
      ):
       return 1
    else:
        return 0

In [33]:
#pass the codes
faults_target_data['is_derate_code'] = faults_target_data.apply(lambda row: is_derate_codes(row, derate_codes), axis=1)

C:\Users\nitin\AppData\Local\Temp\ipykernel_64276\3759071785.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  faults_target_data['is_derate_code'] = faults_target_data.apply(lambda row: is_derate_codes(row, derate_codes), axis=1)


In [34]:
faults_target_data['is_derate_code'].value_counts()

is_derate_code
0    770217
1     50045
Name: count, dtype: int64

### Now loop through the data and grab all diagnostic data for records where ind_derate = 1 per equipment ID. We will use these as predictors. 

In [35]:
faults_target_data

,RecordID,EquipmentID,EventTimeStamp,spn,fmi,active,AcceleratorPedal,BarometricPressure,CruiseControlSetSpeed,EngineCoolantTemperature,...,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,is_derate_code
0,1,1439,2015-02-21 10:47:13,111,17,True,0,14.21,66.48672,100.4,...,NaN,False,78.8,1023,NaN,0,3276.75,NaN,0,0
1,2,1439,2015-02-21 11:34:34,629,12,True,NaN,NaN,NaN,NaN,...,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,0
2,3,1369,2015-02-21 11:35:31,1807,2,False,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN,0
3,4,1369,2015-02-21 11:35:33,1807,2,True,NaN,NaN,NaN,NaN,...,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,0
4,5,1674,2015-02-21 11:39:41,4364,17,False,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,16639,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
908436,1236126,2206,2000-06-08 12:06:28,37,1,True,0,14.355,0,170.6,...,NaN,True,122,50175,NaN,0,NaN,100,0,0
908439,1236129,2206,2000-06-08 12:08:43,37,1,False,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,50175,NaN,NaN,NaN,NaN,NaN,0
909777,1237840,2175,2011-01-01 00:03:22,5246,16,True,0,14.4275,0,120.2,...,NaN,True,93.2,6143,NaN,0,NaN,100,0,0
909778,1237841,2175,2011-01-01 00:03:23,5246,0,True,0,14.4275,0,120.2,...,NaN,True,93.2,6143,NaN,0,NaN,100,0,0


In [36]:
faults_target_data[faults_target_data['EquipmentID'] == 1366].head(10)

,RecordID,EquipmentID,EventTimeStamp,spn,fmi,active,AcceleratorPedal,BarometricPressure,CruiseControlSetSpeed,EngineCoolantTemperature,...,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,is_derate_code
27992,36625,1366,2015-04-28 11:46:22,4794,31,True,0,14.2825,66.48672,188.6,...,NaN,True,116.6,17407,NaN,66.95274,3276.75,NaN,16.53,0
28405,37194,1366,2015-04-28 20:22:08,4794,31,False,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,17407,NaN,NaN,NaN,NaN,NaN,0
28413,37202,1366,2015-04-28 20:26:42,596,31,True,0,14.355,59.65163,190.4,...,NaN,True,89.6,255,NaN,56.27293,3276.75,NaN,15.08,0
28438,37235,1366,2015-04-28 21:27:06,596,31,False,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN,0
33183,43154,1366,2015-05-05 07:01:55,50353,0,True,100,14.5,66.48672,199.4,...,NaN,True,98.6,2,NaN,65.3799,3276.75,NaN,5.51,0
33189,43161,1366,2015-05-05 07:05:10,50353,0,False,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,255,NaN,NaN,NaN,NaN,NaN,0
35111,45520,1366,2015-05-07 04:06:15,4364,18,True,NaN,NaN,NaN,NaN,...,NaN,True,NaN,17407,NaN,NaN,NaN,NaN,NaN,1
39177,50824,1366,2015-05-12 17:58:23,4364,18,False,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1023,NaN,NaN,NaN,NaN,NaN,0
40459,52472,1366,2015-05-14 06:41:15,4364,18,True,0,14.79,66.48672,188.6,...,NaN,True,102.2,17407,NaN,67.02071,3276.75,NaN,11.02,1
41649,54925,1366,2015-05-15 09:57:18,50353,0,True,100,14.7175,66.48672,185,...,NaN,True,116.6,2,NaN,66.99158,3276.75,NaN,10.15,0


In [37]:
faults_target_data = faults_target_data.drop(['LampStatus','ServiceDistance'],axis = 1)

fault_test = faults_target_data.copy()

columns = ['AcceleratorPedal', 'BarometricPressure', 'CruiseControlSetSpeed',
       'EngineCoolantTemperature', 'EngineLoad', 'EngineOilPressure',
       'EngineOilTemperature', 'EngineRpm', 'EngineTimeLtd', 'FuelLevel',
       'FuelLtd', 'FuelTemperature', 'IgnStatus', 'IntakeManifoldTemperature',
       'Speed', 'SwitchedBatteryVoltage', 'Throttle', 'TurboBoostPressure']

for col in columns:

    fault_test[col] = pd.to_numeric(fault_test[col], errors='coerce')


    fault_test[col] = fault_test.groupby('EquipmentID')[col].transform(
        lambda x: x.fillna(x.mean())
    )


    fault_test[col] = fault_test[col].fillna(fault_test[col].mean())

In [38]:
fault_test[fault_test['EquipmentID'] == 1366].head(10)

,RecordID,EquipmentID,EventTimeStamp,spn,fmi,active,AcceleratorPedal,BarometricPressure,CruiseControlSetSpeed,EngineCoolantTemperature,...,FuelLevel,FuelLtd,FuelTemperature,IgnStatus,IntakeManifoldTemperature,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,is_derate_code
27992,36625,1366,2015-04-28 11:46:22,4794,31,True,0.000000,14.28250,66.486720,188.600000,...,63.778889,74868.604999,36.783623,NaN,116.600000,66.952740,3276.75,66.770922,16.530000,0
28405,37194,1366,2015-04-28 20:22:08,4794,31,False,10.156906,14.42544,63.338667,185.122727,...,63.778889,80361.598611,36.783623,NaN,118.738122,29.312686,3276.75,66.770922,6.927045,0
28413,37202,1366,2015-04-28 20:26:42,596,31,True,0.000000,14.35500,59.651630,190.400000,...,63.778889,74916.816399,36.783623,NaN,89.600000,56.272930,3276.75,66.770922,15.080000,0
28438,37235,1366,2015-04-28 21:27:06,596,31,False,10.156906,14.42544,63.338667,185.122727,...,63.778889,80361.598611,36.783623,NaN,118.738122,29.312686,3276.75,66.770922,6.927045,0
33183,43154,1366,2015-05-05 07:01:55,50353,0,True,100.000000,14.50000,66.486720,199.400000,...,63.778889,75161.175547,36.783623,NaN,98.600000,65.379900,3276.75,66.770922,5.510000,0
33189,43161,1366,2015-05-05 07:05:10,50353,0,False,10.156906,14.42544,63.338667,185.122727,...,63.778889,80361.598611,36.783623,NaN,118.738122,29.312686,3276.75,66.770922,6.927045,0
35111,45520,1366,2015-05-07 04:06:15,4364,18,True,10.156906,14.42544,63.338667,185.122727,...,63.778889,80361.598611,36.783623,NaN,118.738122,29.312686,3276.75,66.770922,6.927045,1
39177,50824,1366,2015-05-12 17:58:23,4364,18,False,10.156906,14.42544,63.338667,185.122727,...,63.778889,80361.598611,36.783623,NaN,118.738122,29.312686,3276.75,66.770922,6.927045,0
40459,52472,1366,2015-05-14 06:41:15,4364,18,True,0.000000,14.79000,66.486720,188.600000,...,63.778889,75772.073417,36.783623,NaN,102.200000,67.020710,3276.75,66.770922,11.020000,1
41649,54925,1366,2015-05-15 09:57:18,50353,0,True,100.000000,14.71750,66.486720,185.000000,...,63.778889,75862.156087,36.783623,NaN,116.600000,66.991580,3276.75,66.770922,10.150000,0


In [39]:
no_fd = final_df[(final_df['spn'] != 5246) & (final_df['Hours_Before_5246'] >= 2)]
before = list(no_fd['RecordID'].unique())
fault_test['target'] = fault_test['RecordID'].isin(before).astype(int)
fault_test = fault_test[fault_test['active'] == True]
fault_test

,RecordID,EquipmentID,EventTimeStamp,spn,fmi,active,AcceleratorPedal,BarometricPressure,CruiseControlSetSpeed,EngineCoolantTemperature,...,FuelLtd,FuelTemperature,IgnStatus,IntakeManifoldTemperature,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,is_derate_code,target
0,1,1439,2015-02-21 10:47:13,111,17,True,0.000000,14.210000,66.486720,100.400000,...,12300.907429,36.783623,NaN,78.800000,0.000000,3276.750000,66.770922,0.000000,0,0
1,2,1439,2015-02-21 11:34:34,629,12,True,42.947368,14.484737,66.486720,170.600000,...,14350.152604,36.783623,NaN,107.694737,33.846930,3276.750000,66.770922,6.911667,0,0
3,4,1369,2015-02-21 11:35:33,1807,2,True,32.600000,14.241071,65.554661,188.738462,...,74823.746553,36.783623,NaN,91.014286,34.561692,3276.750000,66.770922,10.585000,0,0
5,6,1417,2015-02-21 11:40:22,111,17,True,48.000000,14.427500,64.622600,181.400000,...,70349.809964,36.783623,NaN,111.200000,13.602200,3276.750000,66.770922,6.670000,0,0
6,8,1429,2015-02-21 11:40:22,111,17,True,60.000000,14.326000,64.249782,187.520000,...,68176.836333,36.783623,NaN,93.560000,47.231978,3276.750000,66.770922,15.370000,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
906287,1232458,2174,2000-05-30 09:00:59,111,4,True,0.000000,14.500000,0.000000,118.400000,...,28540.884326,36.783623,NaN,84.200000,0.000000,3223.594872,100.000000,0.000000,0,0
908436,1236126,2206,2000-06-08 12:06:28,37,1,True,0.000000,14.355000,0.000000,170.600000,...,17626.880170,36.783623,NaN,122.000000,0.000000,3223.594872,100.000000,0.000000,0,0
909777,1237840,2175,2011-01-01 00:03:22,5246,16,True,0.000000,14.427500,0.000000,120.200000,...,30812.896059,36.783623,NaN,93.200000,0.000000,3223.594872,100.000000,0.000000,0,0
909778,1237841,2175,2011-01-01 00:03:23,5246,0,True,0.000000,14.427500,0.000000,120.200000,...,30812.896059,36.783623,NaN,93.200000,0.000000,3223.594872,100.000000,0.000000,0,0


In [40]:
fault_test['code_spn_fmi']= fault_test['spn'].astype(str) +'_'+ fault_test['fmi'].astype(str)
fault_test.to_csv('../data/Final_test_data.csv',index=False)

In [41]:
fault_test['code_spn_fmi'].value_counts()

code_spn_fmi
111_17     132054
929_9       99694
96_3        24872
829_3       23696
111_18      11436
            ...  
4354_4          1
116_31          1
153_7           1
3216_21         1
4607_18         1
Name: count, Length: 882, dtype: int64